In [1]:
import sys
sys.path.append("../") # go to parent dir

In [2]:
import os
import torch
import fairseq
from fairseq.models.bart import BARTModel

from utils import read_lines, construct_multi_label_dataset

2022-06-20 16:17:46 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


In [3]:
# !nvidia-smi

In [4]:
GPU = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = GPU

if torch.cuda.is_available():    
    print(torch.cuda.current_device())
    print(torch.cuda.device)
    torch.cuda.device = GPU
    print(torch.cuda.device)
    device = 'cuda'
else:
    device = 'cpu'
print(f'device: {device}')

0
<class 'torch.cuda.device'>
1
device: cuda


In [5]:
checkpoint_path = '/net/cephfs/scratch/tkew/ctrl_tokens/resources/models/muss_en_mined/model.pt'

def load_bart_checkpoint(checkpoint_path):
    """Checkpoint path should end in model.pt"""
    sd = torch.load(checkpoint_path, map_location="cpu")
    hub_interface = torch.hub.load("pytorch/fairseq", "bart.large.cnn").eval()
    hub_interface.model.load_state_dict(sd["model"])
    return hub_interface

bart = load_bart_checkpoint(checkpoint_path)
bart.model.upgrade_state_dict(bart.model.state_dict())
bart.to(device)

Using cache found in /home/cluster/tkew/.cache/torch/hub/pytorch_fairseq_main
2022-06-20 16:18:06 | INFO | fairseq.file_utils | loading archive file http://dl.fbaipublicfiles.com/fairseq/models/bart.large.cnn.tar.gz from cache at /home/cluster/tkew/.cache/torch/pytorch_fairseq/6ee6af21df382be253b815b53834cd71f61b7ebb83deaa2d10c5d3f94db833cb.3a612b08cd8ff157daf01330f2160f0889831fcb543ec25aa63e38c726167380
2022-06-20 16:18:18 | INFO | fairseq.tasks.translation | [source] dictionary: 50264 types
2022-06-20 16:18:18 | INFO | fairseq.tasks.translation | [target] dictionary: 50264 types


BARTHubInterface(
  (models): ModuleList(
    (0): BARTModel(
      (encoder): TransformerEncoderBase(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(50264, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
        (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0): TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dropout_module): Fair

In [8]:
sentences = read_lines('/net/cephfs/scratch/tkew/ctrl_tokens/resources/data/examples.en')[:1]
sentences, labels = construct_multi_label_dataset(sentences)
print(len(sentences))
print(sentences[0])

100%|██████████| 1/1 [00:00<00:00,  8.40it/s]

*** Constructed 1296 inputs ***
1296
<DEPENDENCYTREEDEPTHRATIO_0.0> <WORDRANKRATIO_0.0> <REPLACEONLYLEVENSHTEIN_0.0> <LENGTHRATIO_0.0> This is extremely hard to comprehend.


In [9]:
tokens = bart.encode(sentences[0]).unsqueeze(0)
print(tokens)

tensor([[    0, 41552, 41372,  9309, 23451,   565,  4629,  1691,  9662,  3732,
           500,  2571,  6454,  1215,   288,     4,   288, 15698, 28696, 42204,
         10644, 10227,   500,  2571,  6454,  1215,   288,     4,   288, 15698,
         28696,  4629,  7205, 15949,  2191, 14079,  3850,   846, 12743, 14469,
           717,  2444,  1215,   288,     4,   288, 15698, 28696,   574, 46219,
           500,  2571,  6454,  1215,   288,     4,   288, 15698,   152,    16,
          2778,   543,     7, 30030,     4,     2]])


In [10]:
gen_outputs = bart.generate(tokens, return_all_hiddens=True)
# return_all_hiddens=True returns attentions, but not hidden layers

In [105]:
print(len(gen_outputs))
print(len(gen_outputs[0]['attention']))
print(gen_outputs[0]['attention'].shape)

1
66
torch.Size([66, 10])


In [ ]:
prev_output_tokens = tokens.clone()

prev_output_tokens[:, 0] = tokens.gather(
    1,
    (tokens.ne(bart.source_dictionary.pad()).sum(dim=1) - 1).unsqueeze(-1),
).squeeze()

prev_output_tokens[:, 1:] = tokens[:, :-1]

In [65]:
print(type(bart))

<class 'fairseq.models.bart.hub_interface.BARTHubInterface'>


In [83]:
outputs = bart.extract_features(tokens, return_all_hiddens=True)

In [85]:
print(len(outputs))
print(outputs[0].shape)
print(outputs[-1].shape)

13
torch.Size([1, 66, 1024])
torch.Size([1, 66, 1024])


In [82]:
prev_output_tokens[:, 1:] = tokens[:, :-1]
output = bart.model(tokens, len(tokens), prev_output_tokens, return_all_hiddens=True)
print(output[1].keys())
len(output[1]['inner_states'])

NameError: name 'prev_output_tokens' is not defined

In [81]:
output[1]['inner_states'][-1].shape

torch.Size([66, 1, 1024])

In [64]:
bart.extract_features(tokens)
# output = bart(tokens)

tensor([[[-0.4098, -0.9986, -1.0965,  ...,  0.8552,  0.1927,  0.2638],
         [-0.4098, -0.9986, -1.0965,  ...,  0.8552,  0.1927,  0.2638],
         [ 1.5466,  0.2715, -0.8765,  ..., -0.7161,  0.0047,  0.8937],
         ...,
         [-0.4998,  1.2425, -1.3456,  ...,  0.1521,  0.9878,  0.5676],
         [-0.5365,  0.5076, -0.6675,  ...,  0.0743,  0.1318,  0.3650],
         [-0.3094,  0.3058, -0.3348,  ...,  0.3210,  0.1646,  0.1003]]],
       grad_fn=<TransposeBackward0>)

In [42]:
output = bart.generate(tokens,
    beam=5,
    hypothesis_num=1,
    lenpen=1.0,
#     diverse_beam_groups=None,
    diverse_beam_strength=0.5,
    sampling=False,
    return_all_hiddens=True,
    )

In [43]:
output

[{'tokens': tensor([    0,   713,    16,   182,   543,     7, 30030,  2778,     4,     2]),
  'score': tensor(-1.8655),
  'attention': tensor([[1.3580e-01, 1.3580e-01, 9.5752e-02, 6.3426e-02, 5.2657e-02, 9.1252e-02,
           2.7162e-02, 4.6830e-02, 9.1995e-02, 7.9162e-02],
          [1.0065e-02, 1.0065e-02, 5.5935e-03, 4.6675e-03, 2.8600e-03, 5.7284e-03,
           1.9275e-03, 2.3170e-03, 4.4790e-03, 1.6496e-03],
          [2.9825e-03, 2.9825e-03, 6.9103e-03, 7.0867e-03, 6.7883e-03, 7.0269e-03,
           3.6054e-03, 4.0714e-03, 3.4280e-03, 1.8403e-03],
          [1.8803e-03, 1.8803e-03, 3.0364e-03, 3.0701e-03, 1.8105e-03, 1.7790e-03,
           1.7726e-03, 2.6965e-03, 2.0019e-03, 2.5350e-03],
          [2.4417e-03, 2.4417e-03, 6.2699e-03, 6.5042e-03, 6.3820e-03, 6.7361e-03,
           3.0720e-03, 3.6680e-03, 3.2310e-03, 1.9298e-03],
          [2.2584e-03, 2.2584e-03, 5.9672e-03, 6.3054e-03, 5.8783e-03, 5.7519e-03,
           2.9713e-03, 3.2486e-03, 3.0587e-03, 1.9769e-03],
         

In [39]:
# print(output)
bart.decode(output[0]['tokens'])

'This is very hard to comprehend extremely.'

In [40]:
? bart.extract_features

Signature:
 bart.extract_features(
    tokens: torch.LongTensor,
    return_all_hiddens: bool = False,
) -> torch.Tensor
Docstring: <no docstring>
File:      ~/.conda/envs/ctrl_tokens/lib/python3.8/site-packages/fairseq/models/bart/hub_interface.py
Type:      method


In [93]:
# model_dir = '/net/cephfs/scratch/tkew/ctrl_tokens/resources/models/muss_en_mined/model.pt'
# bart = BARTModel.from_pretrained(model_dir, checkpoint_file='model.pt')

CompressionError: unknown compression type 'pt'

In [ ]:
translate_model = load_pretrained_bart(checkpoint_dir)
results = translate_model.translate(
        sentences=sequences,
        beam=5
    )
